In [1]:
class wraplist(object):
    def __init__(self, data):
        self.data = data
        self.n = len(data)
    def __getitem__(self, sl):
        if type(sl) == int:
            return self.data[sl]
        elif sl.start is not None and sl.start == sl.stop:
            return wraplist([])
        stop = None if sl.stop is None or sl.stop == self.n else sl.stop % self.n
        if stop is not None and sl.start >= stop:
            return wraplist(self.data[slice(sl.start, None, sl.step)] + self.data[slice(None, stop, sl.step)])
        else:
            return wraplist(self.data[slice(sl.start, stop, sl.step)])        
    def __setitem__(self, sl, val):
        if type(sl) == int:
            self.data[sl] = val
            return
        elif sl.start is not None and sl.start == sl.stop:
            return
        stop = None if sl.stop is None or sl.stop == self.n else sl.stop % self.n
        if stop is not None and sl.start >= stop:
            self.data[slice(sl.start, None, sl.step)] = val[slice(0, self.n - sl.start)]
            self.data[slice(None, stop, sl.step)] = val[slice(self.n - sl.start, None)]
            return
        else:
            self.data[slice(sl.start, stop, sl.step)] = val
            return
    def __len__(self):
        return len(self.data)
    def __repr__(self):
        return self.data.__repr__()

In [2]:
def revert(wl, start, length):
    #print(start, length, wl[start:start+length], wl[start:start+length][::-1])
    wl[start:start+length] = wl[start:start+length][::-1]

In [3]:
# test case
pos = 0
skip = 0
circle = wraplist(list(range(5)))
for l in list(map(int, "3, 4, 1, 5".split(', '))):
    revert(circle, pos, l)
    print(l, pos, circle)
    pos = (pos + l + skip) % len(circle)
    skip += 1
print(circle, circle[0] * circle[1])

3 0 [2, 1, 0, 3, 4]
4 3 [4, 3, 0, 1, 2]
1 3 [4, 3, 0, 1, 2]
5 1 [3, 4, 2, 1, 0]
[3, 4, 2, 1, 0] 12


In [4]:
def cycle(circle, lengths, pos, skip):
    for l in lengths:
        revert(circle, pos, l)
        # print(l, circle)
        pos = (pos + l + skip) % len(circle)
        skip += 1
    return circle, pos, skip

In [5]:
def densen(sparse_hash):
    dense_hash = []
    for i in range(16):
        offset = i * 16
        val = sparse_hash[offset]
        for j in range(1, 16):
            val ^= sparse_hash[offset + j]
        dense_hash.append(val)
    return dense_hash

def print_hash(dh):
    strings = map(lambda x: "{0:02x}".format(x), dh)
    return "".join(strings)

In [6]:
def getHash(string):
    inp = list(map(ord, string))
    inp += [17, 31, 73, 47, 23]
    circle = wraplist(list(range(256)))
    pos = 0
    skip = 0
    for i in range(64):
        circle, pos, skip = cycle(circle, inp, pos, skip)
    dh = densen(circle)
    return print_hash(dh)

In [7]:
assert(getHash('') == 'a2582a3a0e66e6e86e3812dcb672a272')
assert(getHash('AoC 2017') == '33efeb34ea91902bb2f59c9920caa6cd')
assert(getHash('1,2,3') == '3efbe78a8d82f29979031a4aa0b16a9d')
assert(getHash('1,2,4') == '63960835bcdc130f0b66d7ff4f6a5a8e')

In [8]:
def num_squares(inp_str):
    n = 0
    for i in range(128):
        string = inp_str + '-' + str(i)
        hash_str = getHash(string)
        hash_int = int(hash_str, base=16)
        while hash_int > 0:
            if hash_int % 2:
                n += 1
            hash_int >>= 1
    return n    

In [9]:
assert(num_squares('flqrgnkx') == 8108)

In [10]:
# part 1
print(num_squares('hfdlxzhv'))

8230


In [85]:
inp = 'hfdlxzhv'
# test
# inp = 'flqrgnkx'
hashes = list(map(lambda x: getHash(inp + '-' + str(x)), range(128)))
ihashes = list(map(lambda x: int(x, base=16), hashes))
bhashes = list(map(lambda x: format(x, '#0130b')[2:], ihashes))

In [86]:
bhashes

['11001111101001011100100001111010010100111011101110010101111010111111111000000110000100010011100010111001111110100100100110010101',
 '01001001111101010011001100111001111001110010010110000111010010000011011101100000010011110000100010111100000110100001011100011010',
 '11010100100110001001110010001001001101101110011111101001001011110011111100100011001110000010011010100111110010110001010000110100',
 '11111111110111000100000101011111011100100111101101010010010110010001101100010100110010100011011000110110111001100110110101110001',
 '11000011110010100110101010110010100010001010100110000111100100010100010011011001101011010110100101001111101001010101011100000010',
 '11111110011000101110110010000000010000001001101111101111010010101010100101110011110111010000110110100101000000101001110110111110',
 '00111101111000101011100110011111101110000000110000010000111110010010011001010011100000111111011101101010001111100101001110100010',
 '1010011110111101100111100101010001110010011110010000001111110001001

In [87]:
dim = 128

cluster = np.zeros((dim, dim))

for i in range(128):
    for j in range(128):
        if bhashes[i][j] == '1':
            cluster[i, j] = 1

def ngbrs(point):
    if point[0] < dim - 1:
        yield (point[0]+1, point[1]) 
    if point[0] > 0:
        yield (point[0]-1, point[1])
    if point[1] < dim - 1:
        yield (point[0], point[1]+1)
    if point[1] > 0:
        yield (point[0], point[1]-1) 
            
def grow(grid, start):
    stack = [start]
    while len(stack):
        check = stack.pop()
        grid[check] = 0
        for ngb in ngbrs(check):
            if grid[ngb] == 1:
                stack.append(ngb)
                    
ncluster = 0
while np.sum(cluster):
    for i in range(128):
        for j in range(128):
            if cluster[i, j] == 1:
                ncluster += 1
                grow(cluster, (i, j))
print(ncluster)

1103
